In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor 
import time

In [2]:
#Recolhendo dados do facebook e preparando dataset
import pandas_datareader.data as web
import datetime as dt

end = dt.datetime(2020, 12, 31)
start = dt.datetime(2018, 1, 1)

df = web.DataReader("FB", 'yahoo', start, end)

df = df.reset_index()
df = df.drop(columns=['Open','Date','High','Low','Volume','Adj Close'])
df = df.rename(columns={'Close': 'Close 0'})

def window (df, w):
    for i in range(1,w):
        df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
    return df
        
df = window(df,7)
df = df.rename(columns={'Close 0': '+3',
                       'Close 1': '+2',
                       'Close 2': '+1',
                       'Close 3': 'Today',
                       'Close 4': '-1',
                       'Close 5': '-2',
                       'Close 6': '-3'})
df.dropna(inplace=True)
df.head()

,+3,+2,+1,Today,-1,-2,-3
6,187.839996,187.869995,188.279999,186.850006,184.330002,184.669998,181.419998
7,187.770004,187.839996,187.869995,188.279999,186.850006,184.330002,184.669998
8,179.369995,187.770004,187.839996,187.869995,188.279999,186.850006,184.330002
9,178.389999,179.369995,187.770004,187.839996,187.869995,188.279999,186.850006
10,177.600006,178.389999,179.369995,187.770004,187.839996,187.869995,188.279999


In [4]:
#Definindo X e y
X = df.loc[:,['Today','-1','-2','-3']]
y = df.loc[:,['+1','+2','+3']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [9]:
modelo = SVR(C=1.0, epsilon=1e-3, kernel='linear')

In [11]:
modelo_multi = MultiOutputRegressor(modelo)

In [12]:
modelo_multi.fit(X_train, y_train)

MultiOutputRegressor(estimator=SVR(epsilon=0.001, kernel='linear'))

In [13]:
modelo_multi.score(X_test, y_test)

0.9746900207433683

In [21]:
a= X_train[:1]

In [24]:
modelo_multi.predict(a)

array([[204.36036311, 205.13236215, 204.95084381]])

In [27]:
y_train[:1]

,+1,+2,+3
132,203.539993,202.539993,206.919998
